# LCEL 

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [2]:
loader = TextLoader("The_Bhagavad_Gita.txt")
loaded_document = loader.load()

In [3]:
loaded_document

[Document(metadata={'source': 'The_Bhagavad_Gita.txt'}, page_content="The **Bhagavad Gita** is a 700-verse Hindu scripture that is part of the Indian epic *Mahabharata*, specifically in the Bhishma Parva (Book of Bhishma). It is a dialogue between prince **Arjuna** and the god **Krishna**, who serves as his charioteer. The Gita addresses the moral and philosophical dilemmas faced by Arjuna as he prepares to go into battle in the Kurukshetra War, where he is conflicted about fighting against his own relatives, teachers, and friends.\n\n**Historical Context**: The Bhagavad Gita takes place on the battlefield of Kurukshetra, where two sets of cousins, the **Pandavas** and the **Kauravas**, are poised for battle. Arjuna, a warrior prince of the Pandavas, is overwhelmed by doubt and moral confusion about the war. In the midst of this internal conflict, Krishna, who is an incarnation of the divine, offers guidance and wisdom that transcends the immediate situation, focusing on life, duty, an

In [4]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks_of_text = text_splitter.split_documents(loaded_document)

In [5]:
embeddings = OpenAIEmbeddings()

vector_db = FAISS.from_documents(chunks_of_text, embeddings)

In [6]:
vector_db

In [7]:
retriever = vector_db.as_retriever()

In [9]:
response = retriever.invoke("who is Krishna")

In [10]:
response

[Document(metadata={'source': 'The_Bhagavad_Gita.txt'}, page_content='**The Nature of God**: The Bhagavad Gita portrays Krishna as both immanent and transcendent. While he is the charioteer and guide to Arjuna, he also reveals himself as the Supreme Being, beyond all dualities and forms. This divine presence signifies that the Supreme is both within and beyond all things, and true liberation can be attained through devotion to God.'),
 Document(metadata={'source': 'The_Bhagavad_Gita.txt'}, page_content="**Philosophical Insights**: Throughout the Gita, Krishna explains the nature of the material world and the self. He reveals the idea that the universe is governed by the laws of nature and that everything in the world is transient. Understanding this impermanence allows individuals to act with greater wisdom, without attachment to worldly desires. Krishna encourages Arjuna to focus on his higher purpose, which is spiritual liberation rather than material success.\n\n**The Vision of the 

In [11]:
retriever = vector_db.as_retriever(search_kwargs={"k": 2})

In [12]:
#response = retriever.invoke("what did he say about ketanji brown jackson?")

In [13]:
#response


In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [15]:
template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)


model = ChatOpenAI()

In [16]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


In [17]:
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [18]:
response = chain.invoke("Who is Krishna?")

In [19]:
type(response)

str

In [20]:
print("The End")

The End
